# Building Fast Queries on a CSV

The goal of this project is to create a class that represents the laptop inventory provided. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

Here are some queries that we will want to answer:

* Given a laptop id, find the corresponding data.
* Given an amount of money, find whether there are two laptops whose total price is that given amount.
* Identify all laptops whose price falls within a given budget

In [1]:
import csv

with open('laptops.csv', encoding='utf8') as f:
    rows = list(csv.reader(f))
    header = rows[0]
    print(header)
    print(rows[1:6])
    

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'], ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB 

In [2]:
from operator import itemgetter, attrgetter
class Inventory():
   
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='utf8') as f:
            rows = list(csv.reader(f))
            self.header = rows[0]
            self.rows = rows[1:]
            self.id_to_row = {}
            self.prices = set()
            rows_int_price = [x[0:11] + [int(x[12])] for x in self.rows]
            self.rows_by_price = sorted(rows_int_price, key = lambda x:x[-1])
            for row in self.rows:
                self.prices.add(int(row[12]))
                self.id_to_row[row[0]] = row

                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        return self.id_to_row.get(laptop_id)
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if int(row[12]) == dollars:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if int(row1[12]) + int(row2[12]) == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
            if dollars in self.prices:
                    return True
            for price1 in self.prices:
                price2 = dollars - price1
                if price2 in self.prices:
                    return True
            return False
        
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0  
        range_end = len(self.rows_by_price) - 1                       
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:                                          
                range_start = range_middle + 1
        price = self.rows_by_price[range_start][-1]
        
        if price < target_price:                  
            return -1                                      
        return range_start

my_inventory = Inventory('laptops.csv')
print(my_inventory.header)
print(len(my_inventory.rows))
print(my_inventory.get_laptop_from_id_fast('3362737'))
print(my_inventory.get_laptop_from_id_fast('3362736'))


['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303
['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575']
None


Testing time difference between lookup in a list (O(N)) and in a dicionary ~(O(1))

In [3]:
import time
import random 
id_values = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

my_inventory = Inventory('laptops.csv')
total_time_no_dict = 0
for laptop_id in id_values:
    start = time.time()
    my_inventory.get_laptop_from_id(laptop_id)
    end = time.time()
    total_time_no_dict += (end - start)

total_time_dict = 0
for laptop_id in id_values:
    start = time.time()
    my_inventory.get_laptop_from_id_fast(laptop_id)
    end = time.time()
    total_time_dict += (end - start)

print('Time with list(s): ', total_time_no_dict)
print('Time with dictionary(s): ', total_time_dict)

Time with list(s):  1.4296760559082031
Time with dictionary(s):  0.007756471633911133


In [4]:
my_inventory = Inventory('laptops.csv')
print('Is there a gift card for 1000 dollars: ', my_inventory.check_promotion_dollars(1000))
print('Is there a gift card for 442 dollars: ', my_inventory.check_promotion_dollars(442))

Is there a gift card for 1000 dollars:  True
Is there a gift card for 442 dollars:  False


Testing time difference between list and set:

In [5]:
prices = [random.randint(100, 5000) for _ in range(100)]
my_inventory = Inventory('laptops.csv')
total_time_no_set = 0
for price in prices:
    start = time.time()
    my_inventory.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += (end - start)

total_time_set = 0
for price in prices:
    start = time.time()
    my_inventory.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += (end - start)

print('Time with list(s): ', total_time_no_set)
print('Time with set(s): ', total_time_set)

Time with list(s):  7.874443054199219
Time with set(s):  0.001188516616821289


Using binary search to find an laptop that satisfies the required price:

In [6]:
my_inventory = Inventory('laptops.csv')
my_inventory.find_first_laptop_more_expensive(1000)

683

Laptops cheaper than 1000:

In [7]:
my_inventory.rows_by_price[0:683]

[['3564228',
  'Acer',
  'C740-C9QX (3205U/2GB/32GB/Chrome',
  'Netbook',
  '11.6',
  '1366x768',
  'Intel Celeron Dual Core 3205U 1.5GHz',
  '2GB',
  '32GB SSD',
  'Intel HD Graphics',
  'Chrome OS',
  174],
 ['7667029',
  'Asus',
  'Vivobook E200HA',
  'Netbook',
  '11.6',
  '1366x768',
  'Intel Atom x5-Z8350 1.44GHz',
  '2GB',
  '32GB Flash Storage',
  'Intel HD Graphics 400',
  'Windows 10',
  191],
 ['1478754',
  'Vero',
  'V131 (X5-Z8350/4GB/32GB/FHD/W10)',
  'Notebook',
  '13.3',
  'Full HD 1920x1080',
  'Intel Atom X5-Z8350 1.44GHz',
  '4GB',
  '32GB Flash Storage',
  'Intel HD Graphics 400',
  'Windows 10',
  196],
 ['4366200',
  'Asus',
  'E402WA-GA010T (E2-6110/2GB/32GB/W10)',
  'Notebook',
  '14',
  '1366x768',
  'AMD E-Series E2-6110 1.5GHz',
  '2GB',
  '32GB Flash Storage',
  'AMD Radeon R2',
  'Windows 10',
  199],
 ['3840240',
  'Acer',
  'Chromebook C910-C2ST',
  'Notebook',
  '15.6',
  '1366x768',
  'Intel Celeron Dual Core 3205U 1.5GHz',
  '2GB',
  '16GB SSD',
  'Int